In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from models_def import cnn2d,cnn_v1,cnn_v2,cudnn_gru,lstm_v1,cnn_gru,gru_v1
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import text, sequence

model_defines = [
#     [cnn2d,'cnn2d'],
#     [cnn_v1,'cnn_v1'],
#     [cnn_v2,'cnn_v2'],
#     [cudnn_gru,'cudnn_gru'],
     [lstm_v1,'lstm_v1'],
#    [cnn_gru,'cnn_gru'],
#    [gru_v1,'gru_v1']
]

max_features = 180000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [2]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
# 
word_vec_dict = {}
with open('../crawl-300d-2M.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.rstrip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')

EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

(159571, 150) (153164, 150)
tabdublin 64403
calidum 52293
vachss 57615
tulip 34247
port 4885
dell 13376
2000000
Preparing embedding matrix
Null word embeddings: 79399


In [3]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
from keras import backend as K
import pickle
from keras.models import load_model
import gc

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def kf_train(model_func,fold_cnt=3,rnd=1,epo=10,batch=64):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = model_func(maxlen,nb_words,EMBEDDING_DIM,embedding_matrix,trainable_flag=False,comp=True)
        batch_size = batch
        epochs = epo
        file_path="best_model.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
        
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    eval_val(y,train_pred)
    return train_pred, test_pred

rnd_init = 1
for model_func,model_name in model_defines:
    for fold_k in [4]:
        print('==================================')
        if 'cnn' in model_name:
            epo = 10
            batch = 64
        else:
            epo = 8
            batch = 256
        train_pred,test_pred = kf_train(model_func,fold_k,rnd_init,epo,batch)
        output_feat = '../features/fasttext_{}_{}_feat.pkl'.format(model_name,fold_k)
        with open(output_feat,'wb') as fout:
            pickle.dump([train_pred,test_pred],fout)
        print(output_feat,'done')
        rnd_init += 1
        




Train on 119678 samples, validate on 39893 samples
Epoch 1/8
119678/119678 [==============================] - 45s 374us/step - loss: 0.0607 - acc: 0.9789 - val_loss: 0.0448 - val_acc: 0.9824
Epoch 2/8
119678/119678 [==============================] - 45s 372us/step - loss: 0.0432 - acc: 0.9832 - val_loss: 0.0412 - val_acc: 0.9837
Epoch 3/8
119678/119678 [==============================] - 44s 372us/step - loss: 0.0405 - acc: 0.9841 - val_loss: 0.0407 - val_acc: 0.9832
Epoch 4/8
119678/119678 [==============================] - 45s 373us/step - loss: 0.0386 - acc: 0.9848 - val_loss: 0.0403 - val_acc: 0.9839
Epoch 5/8
119678/119678 [==============================] - 45s 373us/step - loss: 0.0369 - acc: 0.9853 - val_loss: 0.0395 - val_acc: 0.9842
Epoch 6/8
119678/119678 [==============================] - 45s 373us/step - loss: 0.0355 - acc: 0.9857 - val_loss: 0.0418 - val_acc: 0.9831
Epoch 7/8
119678/119678 [==============================] - 44s 371us/step - loss: 0.0341 - acc: 0.9862 - val_

Epoch 4/8
119679/119679 [==============================] - 44s 369us/step - loss: 0.0396 - acc: 0.9842 - val_loss: 0.0408 - val_acc: 0.9842
Epoch 5/8
119679/119679 [==============================] - 44s 364us/step - loss: 0.0378 - acc: 0.9849 - val_loss: 0.0409 - val_acc: 0.9840
Epoch 6/8
119679/119679 [==============================] - 43s 363us/step - loss: 0.0365 - acc: 0.9852 - val_loss: 0.0421 - val_acc: 0.9842
Epoch 7/8
119679/119679 [==============================] - 43s 363us/step - loss: 0.0354 - acc: 0.9857 - val_loss: 0.0416 - val_acc: 0.9840
Epoch 8/8
119679/119679 [==============================] - 45s 378us/step - loss: 0.0342 - acc: 0.9861 - val_loss: 0.0407 - val_acc: 0.9845
-------------------------------
0 0.0872311906894 0.966823545632
1 0.0211269456641 0.990875535028
2 0.0432672994378 0.982822693347
3 0.00811856083896 0.997342875585
4 0.058001731293 0.976098413872
5 0.0192062321058 0.993093983243
final 0.0394919933382 0.984509507784
../features/fasttext_lstm_v1_4_fe